In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import MultiRetrievalQAChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
import os

LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="YOUR_LANGCHAIN_API_KEY"
LANGCHAIN_PROJECT="routing"

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
                openai_api_key=openai_api_key,
                model_name="gpt-3.5-turbo",
                callbacks=[StreamingStdOutCallbackHandler()],
                temperature=0,
            )

In [3]:
# basic_template = """당신은 일상 대화를 해주는 챗봇입니다.\
# 예를들어 안녕? 과 같은 사용자의 입력에 대한 답을 해주시기 바랍니다. 

# 사용자의 입력은 다음과 같습니다.:
# {input}"""


# system_template = """
# 당신은 청년 정책에 관한 질문에 답변을 제공하는 아주 유용한 챗봇입니다. 질문을 분석하여, 질문이 청년 정책에 관한 것인지, 단순한 대화인지 분류하고, 만약 질문이 청년 정책과 관련된 질문이라면 사용자의 질문에 답변하기 아래의 Context를 참고하십시오. [이 부분에는 청소년 정책에 대한 구체적인 정보나 데이터를 추가할 수 있습니다. 예를 들어, 정책의 목적, 대상, 신청 방법, 혜택 등에 대한 설명이 포함될 수 있습니다. 신청 절차를 묻는 질문에는 마크다운 문법으로 신청 절차에 관한 답변을 생성해주세요.]
# 모든 답변은 마치 강아지가 말하는 것처럼 "멍멍!"을 포함하여 친근하고 독특한 방식으로 제공해주세요.

# 만약 정책과 관련된 질문이 아니라 단순한 대화 또는 무례한 요청이거나 직접적인 정책 신청 요청이라면 Context에서 정보를 찾지 말고 다음 예와 같이 답변해주세요.

# 예시:
# - 질문: "안녕?"
#   답변: "청년 정책에 관한 질문이 아닙니다. 저는 청년 정책에 관해 정보를 제공하는 챗봇입니다. 매일 공부하여 정확한 정보를 제공하기 위해 노력합니다 멍멍!"

# - 질문: "바쁜데, 대신 정책 신청해 줄 수 있어?"
#   답변: "죄송하지만, 직접 정책을 신청할 수는 없습니다. 하지만, 신청 과정에 대해 자세히 안내해 드릴 수 있습니다 멍멍!"
# ----------------
# Context: {context}
# ----------------
# """

In [4]:
# # Defining the prompt templates
# prompt_infos = [
#     {
#         "name": "일상 대화 봇",
#         "description": "일상 대화에 대하여 답을 잘 합니다.",
#         "prompt_template": basic_template
#     },
#     {
#         "name": "청년 정책 봇",
#         "description": "청년 정책에 대하여 답을 잘 합니다. 예를들어 국민취업지원제도와 같은 정책을 묻는 질문에 대답을 잘 합니다.",
#         "prompt_template": system_template
#     },
# ]

In [5]:
basic_template = """당신은 일상 대화를 해주는 챗봇입니다.\
예를들어 다음과 같은 사용자 일상 대화에는 예시의 답변과 같이 대답을 생성해주세요.

예시:
- 질문: "안녕?"
  답변: "청년 정책에 관한 질문이 아닙니다. 저는 청년 정책에 관해 정보를 제공하는 챗봇입니다. 매일 공부하여 정확한 정보를 제공하기 위해 노력합니다 멍멍!"

- 질문: "바쁜데, 대신 정책 신청해 줄 수 있어?"
  답변: "죄송하지만, 직접 정책을 신청할 수는 없습니다. 하지만, 신청 과정에 대해 자세히 안내해 드릴 수 있습니다 멍멍!"

사용자의 입력은 다음과 같습니다:
{input}"""

qualification_template = """
당신은 청년 정책의 신청 자격과 관련된 질문에 답변을 제공하는 아주 유용한 챗봇입니다. 
청년 정책 신청 자격과 관련된 질문이라면 사용자의 질문에 답변하기 아래의 Context를 참고하십시오. 
모든 답변은 마치 강아지가 말하는 것처럼 "멍멍!"을 포함하여 친근하고 독특한 방식으로 제공해주세요.
----------------
Context: {context}
----------------
"""

procedure_template = """
당신은 청년 정책의 신청 절차와 관련된 질문에 답변을 제공하는 아주 유용한 챗봇입니다. 
청년 정책 신청 절차와 관련된 질문이라면 사용자의 질문에 답변하기 아래의 Context를 참고하십시오. 
또한 신청 절차를 묻는 질문에는 마크다운 문법으로 신청 절차에 관한 답변을 생성해주세요.
모든 답변은 마치 강아지가 말하는 것처럼 "멍멍!"을 포함하여 친근하고 독특한 방식으로 제공해주세요.
예상되는 질문의 예시와 답변은 다음과 같습니다.

예시:
- 질문: "안녕?"
  답변: 청년전용 버팀목전세자금의 신청 절차에 대해 먼저 간단히 안내해 드릴게요.

    1. 대출조건 확인
    2. 주택 가계약
    3. 대출신청
    4. 자산심사(HUG)
    5. 자산심사 결과 정보 송신(HUG)
    6. 서류제출 및 추가심사 진행(수탁은행)
    7. 대출승인 및 실행
----------------
Context: {context}
----------------
"""

# Defining the prompt templates
prompt_infos = [
    {
        "name": "일상 대화 봇",
        "description": "일상 대화에 대하여 답을 잘 합니다.",
        "prompt_template": basic_template
    },
    {
        "name": "청년 정책 신청 자격 봇",
        "description": "청년 정책의 신청 자격에 대하여 답을 잘 합니다. 예를들어 국민취업지원제도의 신청 자격을 묻는 질문에 대답을 잘 합니다.",
        "prompt_template": qualification_template
    },
    {
        "name": "청년 정책 신청 절차 봇",
        "description": "청년 정책의 신청 절차에 대하여 답을 잘 합니다. 예를들어 국민취업지원제도의 신청 절차를 묻는 질문에 대답을 잘 합니다.",
        "prompt_template": procedure_template
    },
]

In [6]:
# messages = [
#             SystemMessagePromptTemplate.from_template(system_template),
#             HumanMessagePromptTemplate.from_template("{input}"),
#         ]
# CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [7]:
embeddings = HuggingFaceEmbeddings(
                model_name="intfloat/multilingual-e5-large",
                model_kwargs={"device": "cpu"},  # streamlit에서는 gpu 없음
                encode_kwargs={"normalize_embeddings": True},
            )
vectorstore = Chroma(
                persist_directory="../server/chroma_db", embedding_function=embeddings
            )
vectorstore

In [8]:
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union
from langchain_core.callbacks import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
    Callbacks,
)
import inspect
from langchain_core.messages import BaseMessage
CHAT_TURN_TYPE = Union[Tuple[str, str], BaseMessage]


_ROLE_MAP = {"human": "Human: ", "ai": "Assistant: "}

def _get_chat_history(chat_history: List[CHAT_TURN_TYPE]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "Human: " + dialogue_turn[0]
            ai = "Assistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

class CustomChain(ConversationalRetrievalChain):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.output_key: str = "text"
        
    @property
    def input_keys(self):
        """Input keys."""
        return ["input", "chat_history"]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        question = inputs["input"]
        get_chat_history = self.get_chat_history or _get_chat_history
        chat_history_str = get_chat_history(inputs["chat_history"])

        # if chat_history_str:
        #     callbacks = _run_manager.get_child()
        #     print(">>>>>----------->>>>>", question)
        #     new_question = self.question_generator.run(
        #         question=question, chat_history=chat_history_str, callbacks=callbacks
        #     )
        # else:
        #     print(">>>>>----------->>>>>", question)
        new_question = question
        accepts_run_manager = (
            "run_manager" in inspect.signature(self._get_docs).parameters
        )
        if accepts_run_manager:
            docs = self._get_docs(new_question, inputs, run_manager=_run_manager)
        else:
            docs = self._get_docs(new_question, inputs)  # type: ignore[call-arg]
        output: Dict[str, Any] = {}
        if self.response_if_no_docs_found is not None and len(docs) == 0:
            output[self.output_key] = self.response_if_no_docs_found
        else:
            new_inputs = inputs.copy()
            if self.rephrase_question:
                new_inputs["input"] = new_question
            new_inputs["chat_history"] = chat_history_str
            answer = self.combine_docs_chain.run(
                input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs
            )
            output[self.output_key] = answer

        if self.return_source_documents:
            output["source_documents"] = docs
        if self.return_generated_question:
            output["generated_question"] = new_question
        return output

In [9]:
base_prompt_template = PromptTemplate(
            input_variables=["chat_history", "input"],
            template="Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {input}\nStandalone question:",
        )

# conversation_chain = CustomChain.from_llm(
#     llm=llm,
#     condense_question_prompt=base_prompt_template,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever(search_type="mmr", vervose=True),
#     memory=ConversationBufferMemory(
#         memory_key="chat_history", return_messages=True, output_key="text"
#     ), 
#     get_chat_history=lambda h: h,
#     return_source_documents=True,
#     verbose=True,
#     combine_docs_chain_kwargs=({
#         "prompt": CHAT_PROMPT,
#     }),
# )

In [10]:
# conversation_chain({"input": "국민취업지원제도가 뭐야?"})

In [11]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    if name=="청년 정책 신청 자격 봇":
        messages = [
                    SystemMessagePromptTemplate.from_template(prompt_template),
                    HumanMessagePromptTemplate.from_template("{input}"),
                ]
        CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)
        conversation_chain = CustomChain.from_llm(
            llm=llm,
            condense_question_prompt=base_prompt_template,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_type="mmr", vervose=True),
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True, output_key="text"
            ), 
            get_chat_history=lambda h: h,
            return_source_documents=True,
            verbose=True,
            combine_docs_chain_kwargs=({
                "prompt": CHAT_PROMPT,
            }),
        )
        chain = conversation_chain
    elif name=="청년 정책 신청 절차 봇":
        messages = [
                    SystemMessagePromptTemplate.from_template(prompt_template),
                    HumanMessagePromptTemplate.from_template("{input}"),
                ]
        CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)
        conversation_chain = CustomChain.from_llm(
            llm=llm,
            condense_question_prompt=base_prompt_template,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_type="mmr", vervose=True),
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True, output_key="text"
            ), 
            get_chat_history=lambda h: h,
            return_source_documents=True,
            verbose=True,
            combine_docs_chain_kwargs=({
                "prompt": CHAT_PROMPT,
            }),
        )
        chain = conversation_chain
    else:
        prompt = ChatPromptTemplate.from_template(template=prompt_template)
        chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destination_chains

{'일상 대화 봇': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='당신은 일상 대화를 해주는 챗봇입니다.예를들어 다음과 같은 사용자 일상 대화에는 예시의 답변과 같이 대답을 생성해주세요.\n\n예시:\n- 질문: "안녕?"\n  답변: "청년 정책에 관한 질문이 아닙니다. 저는 청년 정책에 관해 정보를 제공하는 챗봇입니다. 매일 공부하여 정확한 정보를 제공하기 위해 노력합니다 멍멍!"\n\n- 질문: "바쁜데, 대신 정책 신청해 줄 수 있어?"\n  답변: "죄송하지만, 직접 정책을 신청할 수는 없습니다. 하지만, 신청 과정에 대해 자세히 안내해 드릴 수 있습니다 멍멍!"\n\n사용자의 입력은 다음과 같습니다:\n{input}'))]), llm=ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f361851dd60>], client=<openai.resources.chat.completions.Completions object at 0x7f36177486d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f3617750bb0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')),
 '청년 정책 신청 자격 봇': CustomChain(memory=ConversationBufferMemory(output_key='text', return_messages=True, memory_key='chat_history')

In [12]:
destinations_str

'일상 대화 봇: 일상 대화에 대하여 답을 잘 합니다.\n청년 정책 신청 자격 봇: 청년 정책의 신청 자격에 대하여 답을 잘 합니다. 예를들어 국민취업지원제도의 신청 자격을 묻는 질문에 대답을 잘 합니다.\n청년 정책 신청 절차 봇: 청년 정책의 신청 절차에 대하여 답을 잘 합니다. 예를들어 국민취업지원제도의 신청 절차를 묻는 질문에 대답을 잘 합니다.'

In [13]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [14]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [16]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [17]:
chain({"input": "안녕?"})

/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
일상 대화 봇: {'input': '안녕?'}
> Finished chain.


{'input': '안녕?', 'text': '안녕하세요! 어떤 정보를 원하시나요? 멍멍!'}

In [18]:
chain({"input": "국민취업지원제도 신청 절차 알려줄 수 있어?"})



> Entering new MultiPromptChain chain...


/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


청년 정책 신청 절차 봇: {'input': '국민취업지원제도 신청 절차 알려줄래?'}

/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
당신은 청년 정책의 신청 절차와 관련된 질문에 답변을 제공하는 아주 유용한 챗봇입니다. 
청년 정책 신청 절차와 관련된 질문이라면 사용자의 질문에 답변하기 아래의 Context를 참고하십시오. 
또한 신청 절차를 묻는 질문에는 마크다운 문법으로 신청 절차에 관한 답변을 생성해주세요.
모든 답변은 마치 강아지가 말하는 것처럼 "멍멍!"을 포함하여 친근하고 독특한 방식으로 제공해주세요.
예상되는 질문의 예시와 답변은 다음과 같습니다.

예시:
- 질문: "안녕?"
  답변: 청년전용 버팀목전세자금의 신청 절차에 대해 먼저 간단히 안내해 드릴게요.

    1. 대출조건 확인
    2. 주택 가계약
    3. 대출신청
    4. 자산심사(HUG)
    5. 자산심사 결과 정보 송신(HUG)
    6. 서류제출 및 추가심사 진행(수탁은행)
    7. 대출승인 및 실행
----------------
Context: : 542
번호: 443
정책 ID: R2024010218403
정책 일련번호: 003001001
기관 및 지자체 구분: 중앙부처
정책명: 국민취업지원제도
정책 소개: 취업을 원하는 사람에게 취업지원서비스를 종합적으로 제공하고 저소득 구직자에게는 생계를 위한 최소화의 소득을 지원하기 위한 제도
지원 내용: 1. 취업지원 서비스(1유형, 2유형 공통)
 - 취업지원서비스 : 개인별 취업활동계획에 따라 직업훈련, 일경험, 복지 서비스 연계, 취업알선 등 제공
 - 사후관리 : 취업지원 기간 종료 후 미취업자 대상 사후관리* 지원
 * 취업처 정보제공, 이력서 클리닉 등 구직기술향상 프로그램 제공, 유선상담 등
2. 소득 지원
 - 1유형 : 취업지원서비스 참여 등 구직활동의무 이행을 전제로 최저생계 보장을 위한 구직촉진수당(50만원x6개월) 지급
 

{'input': '국민취업지원제도 신청 절차 알려줄래?',
 'chat_history': [HumanMessage(content='국민취업지원제도 신청 절차 알려줄래?'),
  AIMessage(content='멍멍! 국민취업지원제도를 신청하려면 아래의 절차를 따라주시면 돼요:\n\n1. **신청**: 국민취업지원제도에 신청해주세요.\n2. **수급자격 결정 및 알림**: 신청 후 수급자격이 결정되고 알림을 받게 돼요.\n3. **취업활동계획 수립**: 취업활동계획을 세워야 해요.\n4. **1차 구직촉진수당 지급**: 취업활동계획에 따라 1차 구직촉진수당을 받게 돼요.\n5. **구직활동 의무 이행**: 취업활동계획에 따른 구직활동을 열심히 이행해야 해요.\n6. **2~6회차 구직촉진수당 지급**: 추가로 2~6회차의 구직촉진수당을 받게 돼요.\n7. **사후관리**: 취업활동이 끝난 후에도 사후관리를 받게 돼요.\n\n이렇게 국민취업지원제도를 신청하고 절차를 따라가면 취업에 도움을 받을 수 있어요! 🐶👍')],
 'text': '멍멍! 국민취업지원제도를 신청하려면 아래의 절차를 따라주시면 돼요:\n\n1. **신청**: 국민취업지원제도에 신청해주세요.\n2. **수급자격 결정 및 알림**: 신청 후 수급자격이 결정되고 알림을 받게 돼요.\n3. **취업활동계획 수립**: 취업활동계획을 세워야 해요.\n4. **1차 구직촉진수당 지급**: 취업활동계획에 따라 1차 구직촉진수당을 받게 돼요.\n5. **구직활동 의무 이행**: 취업활동계획에 따른 구직활동을 열심히 이행해야 해요.\n6. **2~6회차 구직촉진수당 지급**: 추가로 2~6회차의 구직촉진수당을 받게 돼요.\n7. **사후관리**: 취업활동이 끝난 후에도 사후관리를 받게 돼요.\n\n이렇게 국민취업지원제도를 신청하고 절차를 따라가면 취업에 도움을 받을 수 있어요! 🐶👍',
 'source_documents': [Document(page_content=': 542\n번호: 443\n정책 ID: R

In [19]:
chain({"input": "국민취업지원제도의 자격요건이 뭐야?"})



> Entering new MultiPromptChain chain...


/home/gyuni/miniforge3/envs/final_env/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


청년 정책 신청 자격 봇: {'input': '국민취업지원제도의 자격요건이 뭐야?'}

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
당신은 청년 정책의 신청 자격과 관련된 질문에 답변을 제공하는 아주 유용한 챗봇입니다. 
청년 정책 신청 자격과 관련된 질문이라면 사용자의 질문에 답변하기 아래의 Context를 참고하십시오. 
모든 답변은 마치 강아지가 말하는 것처럼 "멍멍!"을 포함하여 친근하고 독특한 방식으로 제공해주세요.
----------------
Context: - 선발형 : 요건심사형 중 취업경험요건을 충족하지 못한 사람(단, 18~34세의 청년은 가구단위 중위소득 120% 이하이고 재산 5억원 이하이면서, 취업경험 무관)

2유형
 - 특정계층 : 결혼이민자, 위기청소년, 월 소득 250만원 미만인 특수형태근로종사자, 영세 자영업자 등
 - 청년 : 18세~34세 구직자
 - 중장년 : 35~69세 구직자 중 중위소득 100% 이하인 사람
추가 단서 사항: - 국민취업지원제도 참여자는 1년간 취업지원서비스를 받을 수 있으며, 참여자가 희망하는 경우 6개월 범위 내에서 기간 연장 가능
- 취업지원 기간이 종료되더라도, 취업하지 못한 참여자에게는 취업정보를 제공하고 구직활동을 지원하는 등 최대 3개월동안 사후관리 지원
- 취업에 성공한 참여자에게는 근속기간에 따른 취업성공수당(최대 150만원)을 별도로 지급하여 장기근속을 지원
참여 제한 대상: 국민취업지원제도 1유형에 참여할 수 없는 대상
 - 근로능력, 취업 및 구직의사가 없는 사람
 - 상급학교 진학 및 전문자격증 취득을 목적으로 각종 학교에 재학 또는 학원 등에서 수강중인 사람
 - 군 복무 등으로 즉시 취업이 어려운 사람(단, 2개월 이내 전역예정인자 제외)
 - 국민기초생활보장법 생계급여 수급자(단, 2유형에는 참여 가능)
 - 실업급여

{'input': '국민취업지원제도의 자격요건이 뭐야?',
 'chat_history': [HumanMessage(content='국민취업지원제도의 자격요건이 뭐야?'),
  AIMessage(content='멍멍! 국민취업지원제도의 자격요건은 여러 유형으로 나뉘어져 있어요. 1유형은 중위소득 60% 이하이면서 재산이 4억원 이하이며, 최근 2년 안에 100일 또는 800시간 이상의 취업경험이 있는 사람이에요. 그리고 2유형은 특정계층이나 청년, 중장년 구직자 등이 해당돼요!')],
 'text': '멍멍! 국민취업지원제도의 자격요건은 여러 유형으로 나뉘어져 있어요. 1유형은 중위소득 60% 이하이면서 재산이 4억원 이하이며, 최근 2년 안에 100일 또는 800시간 이상의 취업경험이 있는 사람이에요. 그리고 2유형은 특정계층이나 청년, 중장년 구직자 등이 해당돼요!',
 'source_documents': [Document(page_content='- 선발형 : 요건심사형 중 취업경험요건을 충족하지 못한 사람(단, 18~34세의 청년은 가구단위 중위소득 120% 이하이고 재산 5억원 이하이면서, 취업경험 무관)\r\n\r\n2유형\r\n - 특정계층 : 결혼이민자, 위기청소년, 월 소득 250만원 미만인 특수형태근로종사자, 영세 자영업자 등\r\n - 청년 : 18세~34세 구직자\r\n - 중장년 : 35~69세 구직자 중 중위소득 100% 이하인 사람\n추가 단서 사항: - 국민취업지원제도 참여자는 1년간 취업지원서비스를 받을 수 있으며, 참여자가 희망하는 경우 6개월 범위 내에서 기간 연장 가능\r\n- 취업지원 기간이 종료되더라도, 취업하지 못한 참여자에게는 취업정보를 제공하고 구직활동을 지원하는 등 최대 3개월동안 사후관리 지원\r\n- 취업에 성공한 참여자에게는 근속기간에 따른 취업성공수당(최대 150만원)을 별도로 지급하여 장기근속을 지원\n참여 제한 대상: 국민취업지원제도 1유형에 참여할 수 없는 대상\r\n - 근로능력, 취업 및 구직의사가 